In [205]:
from keras.models import model_from_json

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [230]:
import numpy as np
import cv2

#read the test image
im = cv2.imread('C:/Users/sidhy/OneDrive/Desktop/test.jpg')

#convert image to grayscale format
imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

cv2.imshow("original",im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [231]:
im=~im
#apply binary thresholding
ret,thresh = cv2.threshold(imgray,150,255,cv2.THRESH_BINARY)

cv2.imshow("Binary image",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [232]:
#detect contours on binary image using cv2.CHAIN_APPROX_NONE
contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
#contour retrieval mode used - RETR_TREE - retrieves all of the contours and reconstructs a full hierarchy of nested contours.
#cv2.CHAIN_APPROX_NONE - This will keep all the contour points whenever it runs through any boundary lines. 

print(len(contours))

4


In [233]:
sorted_contours=sorted(contours, key=lambda contour: cv2.boundingRect(contour)[0])

In [234]:
w= int(45)
h= int(45)
train_data= []

In [235]:
rectangles= []
for c in sorted_contours:
    x,y,w,h= cv2.boundingRect(c)
    rect= [x,y,w,h]
    rectangles.append(rect)
    
print(rectangles)

[[0, 0, 357, 436], [91, 67, 156, 263], [95, 184, 148, 142], [97, 71, 122, 110]]


In [236]:
#i[0] = x
#i[1] = y
#i[2] = w
#i[3] = h
bool_rect=[]

for i in rectangles:
    l=[] #list is made empty everytime i value changes
    
    for j in rectangles:
        flag=0 #reset flag to 0 everytime loop runs
        if i!=j:
            if i[0]<(j[0]+j[2]) and j[0]<(i[0]+i[2]) and i[1]<(j[1]+j[3]) and j[1]<(i[1]+i[3]):
                flag=1
            l.append(flag)
        else:
            l.append(0)
        
    bool_rect.append(l) #append this list everytime j loop breaks and i value is going to change.
                        #therefore there will be 'i'no. of lists inside bool_rect, and each list will
                        #have 'j' no. of elements.

print(bool_rect)

[[0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 0]]


In [237]:
discard_rect=[]

for i in range(0, len(sorted_contours)):
    for j in range(0, len(sorted_contours)):
        if bool_rect[i][j] ==1:
            areai = rectangles[i][2]*rectangles[i][3] #w*h
            areaj = rectangles[j][2]*rectangles[j][3]
            if areai == min(areai, areaj):
                discard_rect.append(rectangles[i])
                
#discard_rect will have repeated values
print(discard_rect)

[[91, 67, 156, 263], [95, 184, 148, 142], [95, 184, 148, 142], [97, 71, 122, 110], [97, 71, 122, 110]]


In [238]:
final_rect = [i for i in rectangles if i not in discard_rect]
print(final_rect)

[[0, 0, 357, 436]]


In [239]:
for i in final_rect:
    x = i[0]
    y = i[1]
    w = i[2]
    h = i[3]
    
    im_crop = thresh[y:y+h, x:x+w]
    
    im_resize = cv2.resize(im_crop,(45,45))
    cv2.imshow("image",im_resize)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    im_resize=np.reshape(im_resize,(45,45,1))
    train_data.append(im_resize)

In [240]:
s=''
for i in range(len(train_data)):
    train_data[i]=np.array(train_data[i])
    train_data[i]=train_data[i].reshape(-1,45,45,1)
    result=np.argmax(loaded_model.predict(train_data[i]), axis= -1)
    if(result[0]==0):
        s=s+'+'
    if(result[0]==1):
        s=s+'-'
    if(result[0]==2):
        s=s+'0'
    if(result[0]==3):
        s=s+'1'
    if(result[0]==4):
        s=s+'2'
    if(result[0]==5):
        s=s+'3'
    if(result[0]==6):
        s=s+'4'
    if(result[0]==7):
        s=s+'5'
    if(result[0]==8):
        s=s+'6'
    if(result[0]==9):
        s=s+'7'
    if(result[0]==10):
        s=s+'8'
    if(result[0]==11):
        s=s+'9'
    if(result[0]==12):
        s=s+'='
    if(result[0]==13):
        s=s+'x'
    if(result[0]==14):
        s=s+'cos'
    if(result[0]==15):
        s=s+'/'
    if(result[0]==16):
        s=s+'log'
    if(result[0]==17):
        s=s+'sin'
    if(result[0]==18):
        s=s+'sqrt'
    if(result[0]==19):
        s=s+'tan'
    if(result[0]==20):
        s=s+'*'
    if(result[0]==21):
        s=s+'y'
     

In [241]:
s

'log'